<a href="https://colab.research.google.com/github/laribar/SmartAITraderBot/blob/main/Bot_Funcional_v51104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ta
!pip install yfinance
!pip install xgboost
!pip install python-binance


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=33737a36f57bbdedccfc6b22ef9c61d3e826fd8722b3258bdaad8a50ceffeb6c
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.0 MB/s eta 0:00:00


In [3]:
# ====================================================
# 1. IMPORTAÇÕES
# ====================================================
import yfinance as yf
import numpy as np
import pandas as pd
import ta
import requests
import time  # Para usar time.sleep()
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import pytz
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
XGBClassifier(
    n_estimators=200,
    max_depth=4,                 # menor profundidade = menos overfitting
    subsample=0.8,               # usa 80% dos dados por árvore
    colsample_bytree=0.8,        # usa 80% das features por árvore
    learning_rate=0.05,          # suaviza o aprendizado
    early_stopping_rounds=10,    # para de treinar se não melhorar
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
# ====================================================
# BLOCO 1 - CONFIGURAÇÃO DE PASTAS E IMPORTS EXTRA
# ====================================================
import os
import joblib
from tensorflow.keras.models import load_model

# Criar pasta onde os modelos serão salvos
os.makedirs("/content/models", exist_ok=True)
# ====================================================
# BLOCO 2 - SALVAR E CARREGAR MODELOS TREINADOS
# ====================================================
def get_model_path(asset, interval, model_type="xgb"):
    asset_clean = asset.replace("-", "")
    ext = "joblib" if model_type == "xgb" else "h5"
    return f"/content/models/{model_type}_model_{asset_clean}_{interval}.{ext}"

# --- XGBoost ---
def save_xgb_model(model, asset, interval):
    path = get_model_path(asset, interval, model_type="xgb")
    joblib.dump(model, path)
    print(f"💾 Modelo XGBoost salvo em: {path}")

def load_xgb_model(asset, interval):
    path = get_model_path(asset, interval, model_type="xgb")
    if os.path.exists(path):
        print(f"📂 Modelo XGBoost carregado de: {path}")
        return joblib.load(path)
    return None

# --- LSTM ---
def save_lstm_model(model, asset, interval):
    path = get_model_path(asset, interval, model_type="lstm")
    model.save(path)
    print(f"💾 Modelo LSTM salvo em: {path}")

def load_lstm_model(asset, interval, data=None, window_size=20):
    from tensorflow.keras.models import load_model
    import joblib

    model_path = get_model_path(asset, interval, model_type="lstm")
    meta_path = model_path.replace(".h5", "_meta.pkl").replace(".keras", "_meta.pkl")

    if not os.path.exists(model_path):
        print(f"🚫 Modelo LSTM NÃO encontrado em: {model_path}")
        return None

    try:
        model = load_model(model_path, compile=False)
        print(f"📂 Modelo LSTM encontrado em: {model_path}")
    except Exception as e:
        print(f"❌ Erro ao carregar modelo LSTM de {model_path}: {e}")
        return None

    # ✅ Carrega os metadados salvos em .pkl
    if os.path.exists(meta_path):
        try:
            meta = joblib.load(meta_path)
            model.scaler = meta.get("scaler")
            model.feature_cols = meta.get("feature_cols")
            model.window_size = meta.get("window_size", window_size)
            print(f"📦 Metadados carregados de: {meta_path}")
        except Exception as e:
            print(f"⚠️ Erro ao carregar metadados de {meta_path}: {e}")
            model.scaler = None
            model.feature_cols = None
            model.window_size = window_size
    else:
        print(f"⚠️ Metadados não encontrados em: {meta_path}")
        model.scaler = None
        model.feature_cols = None
        model.window_size = window_size

    return model









# ====================================================
# 2. CONFIGURAÇÕES
# ====================================================
ASSETS = ["BTC-USD"] #, "ETH-USD", "BNB-USD", "SOL-USD", "XRP-USD", "AVAX-USD", "AAVE-USD", "DOT-USD", "NEAR-USD", "ADA-USD"


TIMEFRAMES = [
    {"interval": "15m", "period": "30d", "atr": 0.02},
    {"interval": "1h", "period": "90d", "atr": 0.03},
    {"interval": "1d", "period": "1000d", "atr": 0.05}
]

TELEGRAM_TOKEN = "8044593190:AAFtUWYHd3uqd-AtQi3uqg42F9G6uV95v8k"
TELEGRAM_CHAT_ID = "-4744645054"

# ====================================================
# 3. COLETA DE DADOS
# ====================================================
def get_stock_data(asset, interval="15m", period="700d"):
    data = yf.download(asset, period=period, interval=interval, progress=False, auto_adjust=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [col.split()[-1] if " " in col else col for col in data.columns]
    data = data.loc[:, ~data.columns.duplicated()]
    col_map = {col: std_col for col in data.columns for std_col in ["Open", "High", "Low", "Close", "Adj Close", "Volume"] if std_col.lower() in col.lower()}
    data = data.rename(columns=col_map)
    data = data[["Open", "High", "Low", "Close", "Volume"]]
    if not all(col in data.columns for col in ["Open", "High", "Low", "Close", "Volume"]):
        raise ValueError(f"⚠️ Dados de {asset} não possuem todas as colunas necessárias.")
    return data


def safe_read_csv(filepath):
    import os
    import pandas as pd

    if not os.path.exists(filepath):
        print(f"⚠️ Arquivo não encontrado: {filepath}")
        return None
    if os.path.getsize(filepath) == 0:
        print(f"⚠️ Arquivo está vazio: {filepath}")
        return None
    try:
        df = pd.read_csv(filepath)
        if df.empty or len(df.columns) == 0:
            print(f"⚠️ Arquivo inválido (sem colunas): {filepath}")
            return None
        return df
    except pd.errors.EmptyDataError:
        print(f"⚠️ Erro: arquivo sem colunas: {filepath}")
        return None
    except Exception as e:
        print(f"⚠️ Erro inesperado ao ler CSV: {e}")
        return None

def criar_prediction_log_padrao(filepath="/content/prediction_log.csv"):
    import pandas as pd
    import os

    colunas_padroes = [
        "Asset", "Timeframe", "Date", "Price", "Signal", "Confidence", "AdjustedProb",
        "TP1", "TP2", "SL", "Accuracy", "Precision", "Recall", "F1",
        "LSTM_Predicted", "TargetPrice", "Acertou", "Resultado", "PrecoSaida", "LucroEstimado", "DuracaoMin"
    ]

    if not os.path.exists(filepath) or os.path.getsize(filepath) == 0:
        print(f"📄 Criando novo prediction_log com colunas padrão em: {filepath}")
        df_vazio = pd.DataFrame(columns=colunas_padroes)
        df_vazio.to_csv(filepath, index=False)
    else:
        try:
            df_existente = pd.read_csv(filepath)
            if df_existente.empty or len(df_existente.columns) == 0:
                print(f"⚠️ Arquivo está corrompido. Recriando...")
                df_vazio = pd.DataFrame(columns=colunas_padroes)
                df_vazio.to_csv(filepath, index=False)
            else:
                missing_cols = [col for col in colunas_padroes if col not in df_existente.columns]
                if missing_cols:
                    print(f"⚠️ Adicionando colunas faltantes: {missing_cols}")
                    for col in missing_cols:
                        df_existente[col] = None
                    df_existente.to_csv(filepath, index=False)
        except Exception as e:
            print(f"❌ Erro ao validar o log existente. Recriando. Erro: {e}")
            df_vazio = pd.DataFrame(columns=colunas_padroes)
            df_vazio.to_csv(filepath, index=False)




# ====================================================
# 4. INDICADORES TÉCNICOS
# ====================================================
def calculate_indicators(data):
    data = data.copy().reset_index(drop=True)
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        data[col] = data[col].astype(float)

    # Indicadores Clássicos
    data["RSI"] = ta.momentum.RSIIndicator(close=data["Close"], window=14).rsi()
    data["SMA_50"] = ta.trend.SMAIndicator(close=data["Close"], window=50).sma_indicator()
    data["SMA_200"] = ta.trend.SMAIndicator(close=data["Close"], window=200).sma_indicator()

    macd = ta.trend.MACD(close=data["Close"])
    data["MACD"] = macd.macd()
    data["MACD_Signal"] = macd.macd_signal()

    bb = ta.volatility.BollingerBands(close=data["Close"], window=20)
    data["Bollinger_Upper"] = bb.bollinger_hband()
    data["Bollinger_Lower"] = bb.bollinger_lband()

    adx = ta.trend.ADXIndicator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["ADX"] = adx.adx()

    stoch = ta.momentum.StochasticOscillator(high=data["High"], low=data["Low"], close=data["Close"], window=14)
    data["Stoch_K"] = stoch.stoch()
    data["Stoch_D"] = stoch.stoch_signal()

    # Indicadores adicionais
    data["ATR"] = ta.volatility.AverageTrueRange(high=data["High"], low=data["Low"], close=data["Close"]).average_true_range()
    data["ROC"] = ta.momentum.ROCIndicator(close=data["Close"], window=12).roc()
    data["OBV"] = ta.volume.OnBalanceVolumeIndicator(close=data["Close"], volume=data["Volume"]).on_balance_volume()
    data["CCI"] = ta.trend.CCIIndicator(high=data["High"], low=data["Low"], close=data["Close"], window=20).cci()

    ichimoku = ta.trend.IchimokuIndicator(high=data["High"], low=data["Low"], window1=9, window2=26)
    data["Tenkan_Sen"] = ichimoku.ichimoku_conversion_line()
    data["Kijun_Sen"] = ichimoku.ichimoku_base_line()

    # VWAP e Candles
    data["TP"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["VWAP"] = (data["TP"] * data["Volume"]).cumsum() / (data["Volume"].replace(0, np.nan).cumsum())
    data.drop("TP", axis=1, inplace=True)

    data["Doji"] = ((abs(data["Close"] - data["Open"]) / (data["High"] - data["Low"] + 1e-9)) < 0.1).astype(int)
    data["Engulfing"] = ((data["Open"].shift(1) > data["Close"].shift(1)) & (data["Open"] < data["Close"]) &
                         (data["Close"] > data["Open"].shift(1)) & (data["Open"] < data["Close"].shift(1))).astype(int)
    data["Hammer"] = (((data["High"] - data["Low"]) > 3 * abs(data["Open"] - data["Close"])) &
                      ((data["Close"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6) &
                      ((data["Open"] - data["Low"]) / (data["High"] - data["Low"] + 1e-9) > 0.6)).astype(int)

    data.dropna(inplace=True)
    return data



# ====================================================
# 4. MODELOS DE MACHINE LEARNING (XGBoost + LSTM)
# ====================================================

def get_feature_columns():
    return get_lstm_feature_columns() + ["LSTM_PRED"]

def get_lstm_feature_columns():
    return [
        "Close",  # 🟢 Agora o modelo vai prever o preço de fechamento
        "RSI", "MACD", "MACD_Signal", "SMA_50", "SMA_200", "Bollinger_Upper",
        "Bollinger_Lower", "ADX", "Stoch_K", "Stoch_D", "ATR", "ROC", "OBV", "CCI",
        "Tenkan_Sen", "Kijun_Sen", "VWAP", "Doji", "Engulfing", "Hammer"
    ]


def prepare_lstm_data(data, feature_cols=None, window_size=20):
    if feature_cols is None:
        feature_cols = get_lstm_feature_columns()

    # Verifica se todas as colunas necessárias existem
    missing_cols = [col for col in feature_cols if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Colunas necessárias ausentes: {missing_cols}")

    df = data[feature_cols].copy().dropna().astype(float)
    if len(df) < window_size + 1:
        raise ValueError("Dados insuficientes para preparar dados LSTM")

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(window_size, len(scaled)):
        X.append(scaled[i - window_size:i])
        y.append(scaled[i][0])  # prever a primeira feature

    return np.array(X), np.array(y), scaler

def train_lstm_model(data, asset=None, interval=None, window_size=20, verbose=False, force_retrain=False):
    # Verifica se já existe modelo salvo
    if asset and interval and not force_retrain:
        model = load_lstm_model(asset, interval, data=data, window_size=window_size)
        if model is not None and hasattr(model, "scaler") and model.scaler is not None:
            print(f"✅ LSTM carregado para {asset} ({interval})")
            return model

    if len(data) < window_size + 20:
        print(f"⚠️ Dados insuficientes para treinar LSTM em {asset} ({interval})")
        return None

    print(f"🚀 Treinando LSTM com múltiplas features para {asset} ({interval})...")

    features = get_lstm_feature_columns()
    X, y, scaler = prepare_lstm_data(data, feature_cols=features, window_size=window_size)

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dropout, Dense

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, y, epochs=10, batch_size=32, verbose=0 if not verbose else 1)

    # Atributos do modelo (úteis se for salvar com pickle depois)
    model.scaler = scaler
    model.window_size = window_size
    model.feature_cols = features

    # Salva modelo e metadados
    if asset and interval:
        save_lstm_model(model, asset, interval)

        # ✅ Salva metadados corretamente com joblib
        import joblib
        meta = {
            "scaler": scaler,
            "feature_cols": features,
            "window_size": window_size
        }
        meta_path = get_model_path(asset, interval, model_type="lstm").replace(".h5", "_meta.pkl")
        joblib.dump(meta, meta_path)
        print(f"📦 Metadados salvos em: {meta_path}")

    return model



def train_ml_model(data, asset=None, interval=None, verbose=False):
    if asset and interval:
        existing_model = load_xgb_model(asset, interval)
        if existing_model is not None:
            print(f"✅ Modelo XGBoost já existente para {asset} ({interval}), carregado.")
            return existing_model

    if len(data) < 100:
        return None

    df = data.copy()

    try:
        lstm_model = train_lstm_model(df, asset=asset, interval=interval, force_retrain=False)

        # Verifica se o modelo LSTM foi criado com sucesso
        if lstm_model is not None:
            lstm_preds = []
            for i in range(len(df)):
                sub_df = df.iloc[:i+1]
                if len(sub_df) < lstm_model.window_size:
                    lstm_preds.append(np.nan)
                else:
                    try:
                        pred = predict_with_lstm(lstm_model, sub_df)
                        lstm_preds.append(pred)
                    except Exception as e:
                        print(f"⚠️ Erro ao prever com LSTM: {e}")
                        lstm_preds.append(np.nan)

            # Cria a coluna apenas se tivermos previsões
            if lstm_preds:
                df["LSTM_PRED"] = lstm_preds
        else:
            print("⚠️ Modelo LSTM não foi criado. Continuando sem previsões LSTM.")
            df["LSTM_PRED"] = np.nan

    except Exception as e:
        print(f"⚠️ Erro ao gerar LSTM_PRED: {e}")
        df["LSTM_PRED"] = np.nan

    # Verifica se a coluna foi criada antes de continuar
    if "LSTM_PRED" not in df.columns:
        print("❌ Coluna 'LSTM_PRED' não foi gerada. Abortando treino do XGBoost.")
        return None

    # 🎯 Definir o alvo da previsão
    df["Future_Close"] = df["Close"].shift(-5)
    df["Future_Return"] = df["Future_Close"] / df["Close"] - 1
    df = df[(df["Future_Return"] > 0.015) | (df["Future_Return"] < -0.015)].copy()
    df["Signal"] = np.where(df["Future_Return"] > 0.015, 1, 0)

    features = get_feature_columns()
    df.dropna(inplace=True)

    # Verificação adicional de features
    missing_features = [f for f in features if f not in df.columns]
    if missing_features:
        print(f"❌ Features ausentes: {missing_features}")
        return None

    X = df[features]
    y = df["Signal"]

    if len(np.unique(y)) < 2:
        return None

    from sklearn.model_selection import TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, val_index in tscv.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        break

    if len(np.unique(y_train)) < 2:
        return None

    scale_pos_weight = len(y_train[y_train == 0]) / max(1, len(y_train[y_train == 1]))

    model = XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric="logloss",
        scale_pos_weight=scale_pos_weight,
        random_state=42
    )
    model.fit(X_train, y_train)

    from sklearn.metrics import classification_report
    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, output_dict=True, zero_division=0)

    # ✅ Correção aqui: atribuição feita ao model
    model.validation_score = {
        "accuracy": report.get("accuracy"),
        "precision": report.get("1", {}).get("precision"),
        "recall": report.get("1", {}).get("recall"),
        "f1": report.get("1", {}).get("f1-score")
    }

    if asset and interval:
        save_xgb_model(model, asset, interval)

    return model








def predict_with_lstm(model, data, use_dynamic_calibration=True):
    df = data.copy().reset_index(drop=True)

    if not hasattr(model, "scaler") or model.scaler is None:
        raise ValueError("❌ LSTM carregado sem scaler.")

    if not hasattr(model, "feature_cols"):
        raise ValueError("❌ LSTM carregado sem feature_cols.")

    features = model.feature_cols
    missing = [f for f in features if f not in df.columns]
    if missing:
        raise ValueError(f"❌ Colunas obrigatórias ausentes no dataframe: {missing}")
    df = df[features].dropna().astype(float)

    scaled = model.scaler.transform(df)

    if len(scaled) < model.window_size:
        raise ValueError("❌ Dados insuficientes para previsão com LSTM.")

    last_sequence = scaled[-model.window_size:]
    X_pred = np.reshape(last_sequence, (1, model.window_size, len(features)))
    predicted_scaled = model.predict(X_pred)[0][0]

    # previsão da primeira feature (geralmente Close, RSI, etc.)
    original_value = model.scaler.inverse_transform([[predicted_scaled] + [0] * (len(features) - 1)])[0][0]

    # 🔁 Calibração dinâmica
    if use_dynamic_calibration:
        try:
            df_log = safe_read_csv("/content/prediction_log.csv")
            if df_log is not None:
                df_log = df_log.dropna(subset=["TargetPrice", "Price"])
                ...
            else:
                print("⚠️ Log de previsão indisponível para calibração dinâmica.")

            df_log["Erro"] = df_log["Price"] - df_log["TargetPrice"]
            if len(df_log) >= 10:
                calibration_factor = 1 - (df_log["Erro"].mean() / df_log["TargetPrice"].mean())
                original_value *= calibration_factor
                print(f"🔧 Calibração dinâmica aplicada: fator {calibration_factor:.5f}")
        except Exception as e:
            print(f"⚠️ Erro ao aplicar calibração dinâmica: {e}")

    return round(original_value, 2)




def plot_feature_importance(model, feature_names, top_n=15):
    import matplotlib.pyplot as plt
    importances = model.feature_importances_
    indices = np.argsort(importances)[-top_n:]  # top_n mais importantes

    plt.figure(figsize=(10, 5))
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel("Importância")
    plt.title("🎯 Importância das Features - XGBoost")
    plt.tight_layout()
    plt.grid(True)
    plt.show()



# ====================================================
# 5. UTILITÁRIOS
# ====================================================
# ====================================================
# FUNÇÃO GLOBAL DE CONVERSÃO ESCALAR
# ====================================================
def to_scalar(val):
    try:
        if isinstance(val, pd.Series):
            return float(val.iloc[0])
        elif isinstance(val, (np.ndarray, list)):
            return float(val[0])
        elif pd.isna(val):
            return np.nan
        else:
            return float(val)
    except Exception as e:
        print(f"❌ Falha ao converter valor escalar: {val} | erro: {e}")
        return np.nan

import os
import glob

def limpar_model_results():
    arquivos = glob.glob("/content/model_results_*.csv")
    if not arquivos:
        print("📂 Nenhum arquivo model_results_*.csv encontrado.")
        return


def generate_explanation(row, prediction, feature_importance=None):
    try:
        explicacao = []

        if prediction == 1:
            explicacao.append("🟢 O modelo prevê uma tendência de ALTA.")
        elif prediction == 0:
            explicacao.append("🔴 O modelo prevê uma tendência de BAIXA.")
        else:
            explicacao.append("⚪ Sinal neutro.")

        # Mapas de interpretação técnica
        if row["RSI"] < 30:
            explicacao.append("RSI em sobrevenda (abaixo de 30).")
        elif row["RSI"] > 70:
            explicacao.append("RSI em sobrecompra (acima de 70).")

        if row["MACD"] > row["MACD_Signal"]:
            explicacao.append("MACD cruzando para cima da linha de sinal.")
        else:
            explicacao.append("MACD abaixo da linha de sinal.")

        if row["SMA_50"] > row["SMA_200"]:
            explicacao.append("SMA 50 acima da 200 (tendência de alta).")
        else:
            explicacao.append("SMA 50 abaixo da 200 (tendência de baixa).")

        if row["ADX"] > 20:
            explicacao.append("ADX acima de 20 (tendência definida).")

        if row["Doji"] == 1:
            explicacao.append("Padrão Doji detectado (potencial reversão).")
        if row["Engulfing"] == 1:
            explicacao.append("Padrão de engolfo detectado.")
        if row["Hammer"] == 1:
            explicacao.append("Padrão de candle martelo identificado.")

        if feature_importance:
            top_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:3]
            explicacao.append("\n📊 Principais influências do modelo:")
            for name, weight in top_features:
                explicacao.append(f"• {name}: peso {weight:.3f}")

        return "\n".join(explicacao)

    except Exception as e:
        return f"⚠️ Erro ao gerar explicação: {str(e)}"

def calculate_targets(price, signal, atr_multiplier=0.02):
    atr = price * atr_multiplier
    if signal == 1:
        tp1 = price + atr
        tp2 = price + 2 * atr
        sl = price - atr
    elif signal == -1 or signal == 0:
        tp1 = price - atr
        tp2 = price - 2 * atr
        sl = price + atr
    else:
        return {"TP1": None, "TP2": None, "SL": None}

    return {
        "TP1": round(tp1, 2),
        "TP2": round(tp2, 2),
        "SL": round(sl, 2)
    }





def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "HTML"}
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        print("📨 Mensagem enviada com sucesso!")
    else:
        print(f"❌ Erro ao enviar mensagem: {response.status_code} - {response.text}")

def predict_next_closes(data, n_steps=5):
    df = data.copy().reset_index(drop=True)
    features = get_feature_columns()
    df.dropna(inplace=True)

    X = df[features]
    y = df["Close"].shift(-1).dropna()
    X = X.loc[y.index]

    if len(X) < 100:
        return [None] * n_steps

    model = RandomForestRegressor(n_estimators=200, max_depth=8, random_state=42)
    model.fit(X, y)

    last_row = df[features].iloc[-1].copy()
    preds = []

    for step in range(n_steps):
        X_input = pd.DataFrame([last_row], columns=features)
        next_close = model.predict(X_input)[0]
        preds.append(round(next_close, 2))

        # Simula avanço do mercado
        last_row["Close"] = next_close
        if "SMA_50" in last_row:
            last_row["SMA_50"] = last_row["SMA_50"] * 0.9 + next_close * 0.1
        if "SMA_200" in last_row:
            last_row["SMA_200"] = last_row["SMA_200"] * 0.95 + next_close * 0.05
        if "VWAP" in last_row:
            last_row["VWAP"] = last_row["VWAP"] * 0.95 + next_close * 0.05
        if "RSI" in last_row:
            last_row["RSI"] = min(100, max(0, last_row["RSI"] + np.random.normal(0, 0.5)))
        if "MACD" in last_row:
            last_row["MACD"] += np.random.normal(0, 0.3)
        if "MACD_Signal" in last_row:
            last_row["MACD_Signal"] += np.random.normal(0, 0.2)

        last_row = last_row[features]

    return preds


def evaluate_past_predictions(results_file="/content/prediction_log.csv", lookahead_candles=5):
    import os
    import pandas as pd
    import yfinance as yf
    import matplotlib.pyplot as plt
    from datetime import timedelta

    df = safe_read_csv(results_file)
    if df is None or df.empty:
        print("📭 Nenhum log de previsão encontrado ou o arquivo está vazio.")
        return

    df["Date"] = pd.to_datetime(df["Date"])
    print(f"📊 Avaliando {len(df)} previsões salvas...")

    evaluation = []


    for idx, row in df.iterrows():
        asset = row["Asset"]
        interval = row["Timeframe"]
        prediction_time = row["Date"]
        predicted_signal = row["Signal"]
        predicted_target = row.get("TargetPrice", None)

        try:
            candles = yf.download(asset, start=prediction_time, interval=interval, progress=False)
            candles = candles[candles.index > prediction_time]

            if candles.empty or len(candles) < lookahead_candles:
                continue

            candles = candles.head(lookahead_candles)
            final_close = candles["Close"].iloc[-1]

            if predicted_signal == 1:
                result = "Acertou" if final_close >= predicted_target else "Errou"
            elif predicted_signal == 0:
                result = "Acertou" if final_close <= predicted_target else "Errou"
            else:
                result = "Neutro"

            if predicted_target:
                perc_change = ((final_close - predicted_target) / predicted_target) * 100
                abs_error = final_close - predicted_target
            else:
                perc_change = None
                abs_error = None

            acertou = 1 if result == "Acertou" else 0

            evaluation.append({
                "Ativo": asset,
                "Timeframe": interval,
                "Data Previsão": prediction_time.strftime("%Y-%m-%d %H:%M"),
                "Sinal Previsto": "Compra" if predicted_signal == 1 else "Venda" if predicted_signal == 0 else "Neutro",
                "Valor Projetado (LSTM)": round(predicted_target, 2) if predicted_target else None,
                "Resultado": result,
                "Valor Real": round(final_close, 2),
                "Variação Real": f"{perc_change:+.2f}%" if perc_change is not None else "N/A",
                "Erro Absoluto": f"{abs_error:+.2f}" if abs_error is not None else "N/A",
                "Acertou": acertou
            })

        except Exception as e:
            print(f"⚠️ Erro ao avaliar {asset} em {prediction_time}: {e}")
            continue

    df_eval = pd.DataFrame(evaluation)

    # 📊 Resumo de acertos e erros
    resumo = df_eval.groupby(["Ativo", "Timeframe", "Resultado"]).size().unstack(fill_value=0)
    resumo["Total"] = resumo.sum(axis=1)
    resumo["Acurácia (%)"] = (resumo.get("Acertou", 0) / resumo["Total"] * 100).round(2)
    display(resumo)

    # 📈 Gráfico de barras
    resumo_plot = resumo[["Acertou", "Errou"]] if "Errou" in resumo.columns else resumo[["Acertou"]]
    resumo_plot.plot(kind="bar", figsize=(10, 5), title="📊 Acertos vs Erros por Ativo e Timeframe")
    plt.ylabel("Quantidade de Sinais")
    plt.xticks(rotation=45)
    plt.grid(axis="y")
    plt.tight_layout()
    plt.show()

    # 📄 Tabela completa das previsões
    display(df_eval)

    # 🔄 Atualizar o prediction_log.csv com a coluna 'Acertou'
    try:
        df_log = safe_read_csv(results_file)
        df_log["Date"] = pd.to_datetime(df_log["Date"])

        for _, row in df_eval.iterrows():
            dt = pd.to_datetime(row["Data Previsão"])
            mask = (df_log["Date"] == dt) & (df_log["Asset"] == row["Ativo"]) & (df_log["Timeframe"] == row["Timeframe"])
            df_log.loc[mask, "Acertou"] = row["Acertou"]

        df_log.to_csv(results_file, index=False)
        print("✅ Log de previsões atualizado com coluna 'Acertou'.")

    except Exception as e:
        print(f"❌ Erro ao atualizar o prediction_log.csv com 'Acertou': {e}")

    return df_eval





def clear_models(model_dir="/content/models"):
    import shutil

    if os.path.exists(model_dir):
        print(f"🧹 Limpando todos os modelos salvos em: {model_dir}")
        shutil.rmtree(model_dir)
        os.makedirs(model_dir, exist_ok=True)
        print("✅ Modelos deletados com sucesso.")
    else:
        print("📂 Nenhuma pasta de modelos encontrada para limpar.")


import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_prediction_performance_por_timeframe(log_path="/content/prediction_log.csv"):
    if not os.path.exists(log_path):
        print("📭 Nenhum log encontrado.")
        return

    df = pd.read_csv(log_path)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.dropna(subset=["TargetPrice", "Price", "Timeframe"])

    for timeframe in df["Timeframe"].unique():
        df_tf = df[df["Timeframe"] == timeframe].copy()
        df_tf["Erro"] = df_tf["Price"] - df_tf["TargetPrice"]
        df_tf["AbsError"] = abs(df_tf["Erro"])
        df_tf["Dia"] = df_tf["Date"].dt.date

        if df_tf.empty:
            continue

        # Erro absoluto médio por dia
        plt.figure(figsize=(8, 4))
        df_grouped = df_tf.groupby("Dia")["AbsError"].mean()
        plt.plot(df_grouped.index, df_grouped.values, marker="o")
        plt.title(f"📈 Erro Absoluto Médio por Dia - {timeframe}")
        plt.xlabel("Data")
        plt.ylabel("Erro ($)")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"/tmp/erro_absoluto_{timeframe}.png")
        plt.close()

        # Dispersão do valor previsto x real
        plt.figure(figsize=(8, 4))
        plt.scatter(df_tf["TargetPrice"], df_tf["Price"], alpha=0.6)
        plt.plot([df_tf["TargetPrice"].min(), df_tf["TargetPrice"].max()],
                 [df_tf["TargetPrice"].min(), df_tf["TargetPrice"].max()], 'r--', label="Perfeito")
        plt.title(f"🎯 Previsão LSTM vs Preço Real - {timeframe}")
        plt.xlabel("Valor Previsto")
        plt.ylabel("Valor Real")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        path_img = f"/tmp/previsao_vs_real_{timeframe}.png"
        plt.savefig(path_img)
        plt.close()
        print(f"✅ Gráfico salvo: {path_img}")

def enviar_graficos_desempenho_por_timeframe():
    import glob
    from pathlib import Path

    timeframes = ["15m", "1h", "1d"]  # Edite se tiver outros
    path_base = "/tmp"

    for tf in timeframes:
        # Gráfico 1: Previsão vs Real
        grafico_pred = f"{path_base}/previsao_vs_real_{tf}.png"
        if os.path.exists(grafico_pred):
            with open(grafico_pred, "rb") as img:
                url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                files = {"photo": img}
                data = {
                    "chat_id": TELEGRAM_CHAT_ID,
                    "caption": f"📈 Previsão LSTM vs Real — {tf}"
                }
                r = requests.post(url, data=data, files=files)
                print(f"✅ Enviado: previsao_vs_real_{tf}.png")

        # Gráfico 2: Erro absoluto por dia
        grafico_erro = f"{path_base}/erro_absoluto_{tf}.png"
        if os.path.exists(grafico_erro):
            with open(grafico_erro, "rb") as img:
                url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                files = {"photo": img}
                data = {
                    "chat_id": TELEGRAM_CHAT_ID,
                    "caption": f"📊 Erro Absoluto por Dia — {tf}"
                }
                r = requests.post(url, data=data, files=files)
                print(f"✅ Enviado: erro_absoluto_{tf}.png")

def enviar_grafico_lucro_por_confianca(log_path="/content/prediction_log.csv"):
    import matplotlib.pyplot as plt

    if not os.path.exists(log_path):
        print("📭 Nenhum log encontrado.")
        return

    df = safe_read_csv(log_path)
    if "AdjustedProb" not in df.columns or "TP1" not in df.columns or "Price" not in df.columns:
        print("⚠️ Colunas necessárias não encontradas no log.")
        return

    df = df.dropna(subset=["AdjustedProb", "TP1", "Price"])
    df["LucroEstimado"] = df["TP1"] - df["Price"]
    df["FaixaConfiança"] = pd.cut(df["AdjustedProb"], bins=[0, 0.6, 0.7, 0.8, 0.9, 1.0], labels=["≤60%", "60-70%", "70-80%", "80-90%", ">90%"])

    lucro_medio = df.groupby("FaixaConfiança")["LucroEstimado"].mean()

    plt.figure(figsize=(8, 4))
    lucro_medio.plot(kind="bar", color="skyblue")
    plt.title("📊 Lucro Estimado Médio por Faixa de Confiança")
    plt.ylabel("Lucro Estimado ($)")
    plt.xlabel("Faixa de Confiança Ajustada")
    plt.grid(True)
    plt.tight_layout()

    path = "/tmp/lucro_por_confianca.png"
    plt.savefig(path)
    plt.close()

    with open(path, "rb") as img:
        url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
        files = {"photo": img}
        data = {
            "chat_id": TELEGRAM_CHAT_ID,
            "caption": "📊 Lucro médio estimado por faixa de confiança ajustada"
        }
        response = requests.post(url, data=data, files=files)
        if response.status_code == 200:
            print("✅ Gráfico de lucro por confiança enviado.")
        else:
            print(f"❌ Falha ao enviar gráfico: {response.status_code} - {response.text}")

def adjust_signal_based_on_history(asset, timeframe, max_lookback=20, min_signals=5):
    try:
        df = safe_read_csv("prediction_log.csv")
        if df is None:
            print("⚠️ Ignorando leitura do prediction_log.csv pois está vazio ou ausente.")
            return 1.0  # Retorna confiança padrão

        df["Date"] = pd.to_datetime(df["Date"])
        df = df[(df["Asset"] == asset) & (df["Timeframe"] == timeframe)]

        if len(df) < min_signals or "Acertou" not in df.columns:
            return 1.0

        recent = df.sort_values("Date", ascending=False).head(max_lookback)
        acuracia = recent["Acertou"].mean()
        return acuracia

    except Exception as e:
        print(f"⚠️ Erro ao ajustar com histórico: {e}")
        return 1.0

def gerar_grafico_previsao_vs_real(log_path="/content/prediction_log.csv", output_path="/tmp/previsao_vs_real.png"):
    import matplotlib.pyplot as plt

    df = safe_read_csv(log_path)
    if df is None or df.empty or "TargetPrice" not in df.columns or "Price" not in df.columns:
        print("⚠️ Log inválido ou colunas ausentes.")
        return None

    df = df.dropna(subset=["TargetPrice", "Price"]).tail(20)  # últimos 20 sinais
    df["Date"] = pd.to_datetime(df["Date"])

    plt.figure(figsize=(10, 4))
    plt.plot(df["Date"], df["Price"], label="📈 Preço Real", marker="o")
    plt.plot(df["Date"], df["TargetPrice"], label="🔮 Previsão LSTM", marker="x")
    plt.title("📊 Previsão LSTM vs Preço Real")
    plt.xlabel("Data")
    plt.ylabel("Preço")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()
    print(f"✅ Gráfico salvo em: {output_path}")
    return output_path

def enviar_grafico_previsao_real(df, timeframe):
    import matplotlib.pyplot as plt
    import os

    plt.figure(figsize=(10, 5))
    plt.plot(df["Date"], df["TargetPrice"], label="LSTM Previsto", marker="o")
    plt.plot(df["Date"], df["Price"], label="Preço Real", marker="x")
    plt.title(f"📈 Previsão LSTM vs Preço Real ({timeframe})")
    plt.xlabel("Data")
    plt.ylabel("Preço")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    image_path = f"/tmp/previsao_vs_real_{timeframe}.png"
    plt.savefig(image_path)
    plt.close()

    with open(image_path, "rb") as img:
        url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
        files = {"photo": img}
        data = {
            "chat_id": TELEGRAM_CHAT_ID,
            "caption": f"📉 Previsão LSTM vs Preço Real ({timeframe})"
        }
        response = requests.post(url, data=data, files=files)
        if response.status_code == 200:
            print("✅ Gráfico de previsão enviado.")
        else:
            print(f"❌ Erro ao enviar gráfico: {response.status_code} - {response.text}")



def enviar_grafico_carteira():
    image_path = "/tmp/evolucao_carteira.png"
    if os.path.exists(image_path):
        with open(image_path, "rb") as img:
            url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
            files = {"photo": img}
            data = {
                "chat_id": TELEGRAM_CHAT_ID,
                "caption": "💼 Evolução da carteira virtual com base nos sinais do bot"
            }
            response = requests.post(url, data=data, files=files)
            if response.status_code == 200:
                print("✅ Gráfico da carteira enviado ao Telegram.")
            else:
                print(f"❌ Erro ao enviar imagem: {response.status_code} - {response.text}")



# ====================================================
# 5.1 CARTEIRA VIRTUAL PARA SIMULAÇÃO
# ====================================================
# ====================================================
# 5.1 CARTEIRA VIRTUAL PARA SIMULAÇÃO
# ====================================================

carteira_virtual = {
    "capital_inicial": 10000.0,
    "capital_atual": 10000.0,
    "em_operacao": False,
    "ativo": None,
    "entrada": 0.0,
    "quantidade": 0.0,
    "tipo": None,
    "data_entrada": None
}

def to_scalar(val):
    try:
        if isinstance(val, pd.Series):
            return float(val.iloc[0])
        elif isinstance(val, (np.ndarray, list)):
            return float(val[0])
        elif val is None:
            return np.nan
        else:
            return float(val)
    except Exception as e:
        print(f"❌ Falha ao converter valor escalar: {val} | erro: {e}")
        return np.nan

from datetime import timedelta
import pytz

from datetime import timedelta
import pytz

def simular_trade(row, df):
    try:
        asset = row["Asset"]
        timeframe = row["Timeframe"]
        signal_time = pd.to_datetime(row["Date"])
        preco_entrada = float(row["Price"])
        tp1 = float(row["TP1"])
        sl = float(row["SL"])

        # ✅ Ajusta timezone do signal_time para UTC se necessário
        if df.index.tz is not None:
            if signal_time.tzinfo is None:
                signal_time = signal_time.replace(tzinfo=pytz.UTC)
            else:
                signal_time = signal_time.astimezone(pytz.UTC)

        # 🔍 Seleciona candles futuros
        df_future = df[df.index >= signal_time]
        if df_future.empty or "Close" not in df_future.columns:
            raise ValueError("Candles futuros indisponíveis ou incompletos.")

        for i, (idx, candle) in enumerate(df_future.iterrows()):
            preco_max = float(candle["High"])
            preco_min = float(candle["Low"])

            if row["Signal"] == 1:
                # Trade de COMPRA
                if preco_min <= sl:
                    resultado = "SL"
                    preco_saida = sl
                    break
                elif preco_max >= tp1:
                    resultado = "TP1"
                    preco_saida = tp1
                    break
            else:
                # Trade de VENDA
                if preco_max >= sl:
                    resultado = "SL"
                    preco_saida = sl
                    break
                elif preco_min <= tp1:
                    resultado = "TP1"
                    preco_saida = tp1
                    break
        else:
            resultado = "Sem alvo"
            preco_saida = df_future["Close"].iloc[-1]

        lucro = preco_saida - preco_entrada if row["Signal"] == 1 else preco_entrada - preco_saida
        duracao = (idx - signal_time).total_seconds() / 60

        return {
            "Resultado": resultado,
            "PrecoSaida": preco_saida,
            "LucroEstimado": round(lucro, 2),
            "DuracaoMin": round(duracao, 1)
        }

    except Exception as e:
        print(f"❌ Erro inesperado na simulação: {e}")
        return {
            "Resultado": "Erro",
            "PrecoSaida": None,
            "LucroEstimado": None,
            "DuracaoMin": None
        }

def plotar_grafico_lucro(df):
    import matplotlib.pyplot as plt

    df_valid = df[df["Resultado"].isin(["TP1", "SL", "Sem alvo"])].copy()
    if df_valid.empty:
        print("⚠️ Nenhum resultado válido para gráfico.")
        return

    df_valid["FaixaConfiança"] = pd.cut(
        df_valid["AdjustedProb"].fillna(0.5),
        bins=[0, 0.6, 0.75, 0.9, 1.01],
        labels=["<60%", "60-75%", "75-90%", ">90%"]
    )

    lucro_medio = df_valid.groupby("FaixaConfiança")["LucroEstimado"].mean()

    plt.figure(figsize=(8, 5))
    lucro_medio.plot(kind="bar", color="skyblue", edgecolor="black")
    plt.title("📊 Lucro Médio por Faixa de Confiança")
    plt.ylabel("Lucro Estimado")
    plt.xlabel("Faixa de Confiança")
    plt.grid(True)
    plt.tight_layout()

    path = "lucro_por_faixa.png"
    plt.savefig(path)
    plt.close()
    print("✅ Gráfico de lucro por confiança enviado.")






def simular_todos_trades(prediction_log_path="prediction_log.csv", df_candles=None, timeframe="15m"):
    print("📊 Rodando simulação de carteira virtual com sinais do log...")

    if not os.path.exists(prediction_log_path):
        print("⚠️ Log de previsões não encontrado.")
        return

    df_log = safe_read_csv(prediction_log_path)
    if df_log is None or df_log.empty:
        print("⚠️ Log vazio.")
        return

    if df_candles is None or df_candles.empty:
        if not isinstance(df_candles.index, pd.DatetimeIndex):
            df_candles.index = pd.to_datetime(df_candles.index)


    df_log["Date"] = pd.to_datetime(df_log["Date"])
    resultados = []

    intervalo_futuro = {
        "15m": timedelta(minutes=15 * 5),
        "1h": timedelta(hours=5),
        "4h": timedelta(hours=4 * 5),
        "1d": timedelta(days=5)
    }.get(timeframe, timedelta(hours=1))

    now = datetime.utcnow()

    for _, row in df_log.iterrows():
        signal_time = pd.to_datetime(row["Date"])
        if (now - signal_time) < intervalo_futuro:
            continue  # pula sinais muito recentes

        print("\n🚨 Entrando na função simular_trade()")
        print("🔍 Linha completa:")
        print(row)

        try:
            resultado = simular_trade(row, df_candles)
            for key, value in resultado.items():
                row[key] = value
            resultados.append(row)
        except Exception as e:
            print(f"❌ Erro inesperado na simulação: {e}")
            continue

    if not resultados:
        print("📭 Nenhum trade foi simulado (ainda).")
        return

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(prediction_log_path, index=False)
    print(f"📋 Log de previsões atualizado com resultados: {prediction_log_path}")
    plotar_grafico_lucro(df_resultados)







def salvar_grafico_evolucao():
    import pandas as pd
    import os
    import matplotlib.pyplot as plt

    if not os.path.exists("trades_simulados.csv"):
        print("❌ Arquivo de simulação não encontrado.")
        return

    try:
        df = safe_read_csv("trades_simulados.csv")
        if df.empty or "Data Entrada" not in df.columns:
            print("⚠️ Arquivo vazio ou colunas ausentes.")
            return

        df["Data Entrada"] = pd.to_datetime(df["Data Entrada"], errors="coerce")
        df = df[df["Data Entrada"].dt.year >= 2000]
        if df.empty:
            print("⚠️ Nenhum dado recente disponível para o gráfico.")
            return

        # Escolhe a cor de cada ponto com base no resultado do trade
        cor_map = {
            "TP1": "green",
            "SL": "red",
            "Sem alvo": "orange"
        }
        cores = df["Resultado"].map(cor_map).fillna("gray")

        plt.figure(figsize=(10, 5))
        plt.scatter(df["Data Entrada"], df["Capital Atual"], c=cores, s=70, label="Capital", edgecolors='black')
        plt.plot(df["Data Entrada"], df["Capital Atual"], linestyle="--", alpha=0.5)
        plt.title("💰 Evolução da Carteira Virtual")
        plt.xlabel("Data")
        plt.ylabel("Capital ($)")
        plt.grid(True)
        plt.tight_layout()
        image_path = "/tmp/evolucao_carteira.png"
        plt.savefig(image_path)
        plt.close()
        print("✅ Gráfico salvo em:", image_path)

    except pd.errors.EmptyDataError:
        print("⚠️ trades_simulados.csv está vazio.")


# ====================================================
# 6. EXECUÇÃO DAS ANÁLISES E ALERTAS
# ====================================================

def run_analysis(
    selected_timeframes=None,
    plot_timeframes=["15m", "1h"],
    alert_timeframes=["15m", "1h", "1d"],
    retrain_models=False
):
    criar_prediction_log_padrao()

    if selected_timeframes is None:
        selected_timeframes = TIMEFRAMES

    results = []
    houve_alerta = False

    for asset in ASSETS:
        print(f"\n📊 Analisando {asset}...")
        models = {}
        lstm_models = {}
        data = {}

        try:
            for tf in selected_timeframes:
                interval = tf['interval']
                period = tf['period']
                data[interval] = calculate_indicators(get_stock_data(asset, interval, period))

                if retrain_models:
                    models[interval] = train_ml_model(data[interval], asset=asset, interval=interval, verbose=True)
                else:
                    models[interval] = load_xgb_model(asset, interval)
                    if models[interval] is None:
                        models[interval] = train_ml_model(data[interval], asset=asset, interval=interval, verbose=True)

                lstm_models[interval] = train_lstm_model(
                    data[interval], asset=asset, interval=interval, window_size=20, force_retrain=retrain_models
                )

        except Exception as e:
            print(f"❌ Erro ao processar {asset}: {e}")
            continue

        if all(model is None for model in models.values()):
            print(f"⚠️ Nenhum modelo foi treinado para {asset}.")
            continue

        for tf in selected_timeframes:
            interval = tf['interval']
            latest_data = data[interval].iloc[-1]
            current_price = data[interval]["Close"].iloc[-1]

            predicted_price_lstm = None
            try:
                lstm_model = lstm_models.get(interval)
                if lstm_model:
                    predicted_price_lstm = predict_with_lstm(lstm_model, data[interval])
            except Exception as e:
                print(f"[!] Erro na previsão LSTM: {e}")

            print(f"🔍 Preço atual ({interval}): ${current_price:,.2f}")
            if predicted_price_lstm is not None and not np.isnan(predicted_price_lstm):
                variation = round((predicted_price_lstm - current_price) / current_price * 100, 2)
                print(f"🔮 Previsão LSTM: ${predicted_price_lstm:,.2f}")
                print(f"📈 Variação prevista: {variation:+.2f}%")
            else:
                print("🔮 Previsão LSTM: Indisponível ou inválida.")
                variation = 0.0

            if predicted_price_lstm and not np.isnan(predicted_price_lstm):
                if predicted_price_lstm > current_price * 1.005:
                    prediction = 1
                elif predicted_price_lstm < current_price * 0.995:
                    prediction = 0
                else:
                    prediction = -1
            else:
                prediction = -1

            targets = calculate_targets(current_price, prediction, tf['atr'])
            print("🛠️ DEBUG targets:", targets, type(targets))

            explanation = generate_explanation(latest_data, prediction)

            rr_ratio = round((targets['TP1'] - current_price) / (current_price - targets['SL']), 2) if prediction == 1 else \
                       round((current_price - targets['TP1']) / (targets['SL'] - current_price), 2) if prediction == 0 else "-"

            ajuste = adjust_signal_based_on_history(asset, interval)
            model_xgb = models.get(interval)
            val_score = model_xgb.validation_score if model_xgb and hasattr(model_xgb, "validation_score") else {}

            result = {
                "Asset": asset,
                "Timeframe": interval,
                "Date": datetime.now(),
                "Price": current_price,
                "Signal": prediction,
                "Confidence": None,
                "AdjustedProb": round(ajuste, 2),
                "TP1": targets['TP1'],
                "TP2": targets['TP2'],
                "SL": targets['SL'],
                "Accuracy": val_score.get("accuracy"),
                "Precision": val_score.get("precision"),
                "Recall": val_score.get("recall"),
                "F1": val_score.get("f1"),
                "LSTM_Predicted": predicted_price_lstm,
                "TargetPrice": predicted_price_lstm
            }
            results.append(result)

            if interval in alert_timeframes:
                if prediction in [0, 1]:
                    # ✅ Criação da mensagem dentro do bloco
                    trend_emoji = "🟢" if prediction == 1 else "🔴"
                    trend_text = "COMPRA" if prediction == 1 else "VENDA"

                    message = f"""
📢 <b>SINAL DETECTADO</b>

🪙 <b>Ativo:</b> {asset}
🕒 <b>Timeframe:</b> {interval}
{trend_emoji} <b>Tendência (via LSTM):</b> {trend_text}

💰 <b>Preço Atual:</b> ${current_price:,.2f}
🔮 <b>Projeção LSTM:</b> ${predicted_price_lstm:,.2f} ({variation:+.2f}%)
🎯 <b>TP1:</b> ${targets['TP1']:,.2f}
🎯 <b>TP2:</b> ${targets['TP2']:,.2f}
🛑 <b>Stop Loss:</b> ${targets['SL']:,.2f}

📋 <b>Justificativa Técnica:</b>
{explanation}

📊 <b>Risco/Retorno estimado:</b> {rr_ratio}
🧠 <b>Confiança histórica:</b> {ajuste*100:.1f}%
🗓 <b>Válido até:</b> {(datetime.now() + timedelta(minutes=15)).strftime('%d/%m %H:%M')}
"""

                    try:
                        send_telegram_message(message)
                        print("📨 Alerta enviado para o Telegram!")
                    except Exception as e:
                        print(f"❌ Erro ao enviar mensagem: {e}")

                    try:
                        df_log = safe_read_csv("/content/prediction_log.csv")
                        if df_log is not None:
                            df_log = df_log[df_log["Timeframe"] == interval]
                            df_log = df_log.dropna(subset=["TargetPrice", "Price"])
                            df_log["Date"] = pd.to_datetime(df_log["Date"])
                            df_recent = df_log.sort_values("Date").tail(20)
                            enviar_grafico_previsao_real(df_recent, interval)
                    except Exception as e:
                        print(f"⚠️ Erro ao enviar gráfico de previsão: {e}")

                    try:
                        salvar_grafico_evolucao()
                        enviar_grafico_carteira()
                    except Exception as e:
                        print(f"⚠️ Erro ao enviar gráfico da carteira: {e}")

                    houve_alerta = True
                else:
                    print("⛔ Previsão LSTM neutra — alerta não enviado.")
            else:
                print("⛔ Timeframe fora da lista de alertas.")

    df_results = pd.DataFrame(results)
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"model_results_{timestamp}.csv"
    df_results.to_csv(filename, index=False)
    print(f"\n📁 Resultados salvos em: {filename}")

    log_path = "/content/prediction_log.csv"
    df_log_old = safe_read_csv(log_path)

    if df_log_old is not None:
        common_cols = list(set(df_log_old.columns) & set(df_results.columns))
        df_results = df_results[common_cols]
        df_log_combined = pd.concat([df_log_old, df_results], ignore_index=True)
        df_log_combined.to_csv(log_path, index=False)
        print(f"📋 Log de previsões atualizado em: {log_path}")
    else:
        df_results.to_csv(log_path, index=False)
        print(f"🔃 Log de previsões criado em: {log_path}")

    print("✅ Análise completa.")

    try:
        df_log = safe_read_csv(log_path)
        if df_log is not None and not df_log.empty:
            colunas = ["Date", "Asset", "Timeframe", "Price", "Signal", "TP1", "TP2", "SL", "TargetPrice", "LSTM_Predicted", "AdjustedProb"]
            colunas_disponiveis = [c for c in colunas if c in df_log.columns]
            df_log = df_log[colunas_disponiveis].tail(5)
            print("\n🧪 Últimos sinais registrados no log:")
            print(df_log.to_string(index=False))
        else:
            print("📝 Log de previsões está vazio ou inválido.")
    except Exception as e:
        print(f"⚠️ Erro ao exibir últimos sinais: {e}")




# ====================================================
# 7. AGENDAMENTO E EXECUÇÃO AUTOMÁTICA
# ====================================================

def is_time_to_run(interval):
    now = datetime.now()
    if interval == "15m":
        return now.minute % 15 == 0
    elif interval == "1h":
        return now.minute == 0
    elif interval == "1d":
        return now.hour == 8 and now.minute == 0
    return False

# 🚀 Execução contínua: Verifica os timeframes a cada minuto
while True:
    now = datetime.now()
    print(f"\n⏰ Verificando timeframes - {now.strftime('%Y-%m-%d %H:%M:%S')}")

    for tf in TIMEFRAMES:
        interval = tf["interval"]
        if is_time_to_run(interval):
            print(f"\n🚀 Rodando análise para timeframe {interval}...")
            try:
                run_analysis(
                    selected_timeframes=[tf],
                    plot_timeframes=["1h"],           # Gráficos apenas para timeframes desejados
                    alert_timeframes=["15m", "1h", "1d"]
                )
            except Exception as e:
                print(f"❌ Erro durante a análise de {interval}: {e}")
        else:
            print(f"⏳ Ainda não é hora para {interval}...")

    time.sleep(60)  # Espera 1 minuto antes de verificar de novo


⏰ Verificando timeframes - 2025-04-13 20:29:36
⏳ Ainda não é hora para 15m...
⏳ Ainda não é hora para 1h...
⏳ Ainda não é hora para 1d...

⏰ Verificando timeframes - 2025-04-13 20:30:36

🚀 Rodando análise para timeframe 15m...
📄 Criando novo prediction_log com colunas padrão em: /content/prediction_log.csv

📊 Analisando BTC-USD...
📂 Modelo XGBoost carregado de: /content/models/xgb_model_BTCUSD_15m.joblib
📂 Modelo LSTM encontrado em: /content/models/lstm_model_BTCUSD_15m.h5
📦 Metadados carregados de: /content/models/lstm_model_BTCUSD_15m_meta.pkl
✅ LSTM carregado para BTC-USD (15m)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step
⚠️ Arquivo inválido (sem colunas): /content/prediction_log.csv
⚠️ Log de previsão indisponível para calibração dinâmica.
⚠️ Erro ao aplicar calibração dinâmica: 'NoneType' object is not subscriptable
🔍 Preço atual (15m): $84,202.06
🔮 Previsão LSTM: $84,512.51
📈 Variação prevista: +0.37%
🛠️ DEBUG targets: {'TP1': np.float64(82518.02), 'TP2': np.float64(80833.98), 'SL': n

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
🔍 Preço atual (15m): $83,187.99
🔮 Previsão LSTM: $83,798.14
📈 Variação prevista: +0.73%
🛠️ DEBUG targets: {'TP1': np.float64(84851.75), 'TP2': np.float64(86515.51), 'SL': np.float64(81524.23)} <class 'dict'>
📨 Mensagem enviada com sucesso!
📨 Alerta enviado para o Telegram!
✅ Gráfico de previsão enviado.
❌ Arquivo de simulação não encontrado.

📁 Resultados salvos em: model_results_2025-04-13_21-15-48.csv
📋 Log de previsões atualizado em: /content/prediction_log.csv
✅ Análise completa.

🧪 Últimos sinais registrados no log:
                      Date   Asset Timeframe        Price  Signal      TP1      TP2       SL  TargetPrice  LSTM_Predicted  AdjustedProb
2025-04-13 20:30:37.591004 BTC-USD       15m 84202.062500      -1 82518.02 80833.98 85886.10     84512.51        84512.51           1.0
2025-04-13 20:45:38.891842 BTC-USD       15m 83475.937500       1 85145.46 86814.98 81806.42     84335.29        84335.29           1.0
2025-04-13 21:00:41.766560

KeyboardInterrupt: 

In [22]:
run_analysis(
    selected_timeframes=[{"interval": "15m", "period": "30d", "atr": 0.02}],
    plot_timeframes=["15m"],
    alert_timeframes=["15m"]
)




📊 Analisando BTC-USD...
📂 Modelo XGBoost carregado de: /content/models/xgb_model_BTCUSD_15m.joblib
📂 Modelo LSTM encontrado em: /content/models/lstm_model_BTCUSD_15m.h5
📦 Metadados carregados de: /content/models/lstm_model_BTCUSD_15m_meta.pkl
✅ LSTM carregado para BTC-USD (15m)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
🔍 Preço atual (15m): $84,957.06
🔮 Previsão LSTM: $83,296.71
📈 Variação prevista: -1.95%
🛠️ DEBUG targets: {'TP1': np.float64(83257.92), 'TP2': np.float64(81558.78), 'SL': np.float64(86656.2)} <class 'dict'>
📨 Mensagem enviada com sucesso!
📨 Alerta enviado para o Telegram!
✅ Gráfico de previsão enviado.
❌ Arquivo de simulação não encontrado.

📊 Analisando ETH-USD...
🚫 Modelo LSTM NÃO encontrado em: /content/models/lstm_model_ETHUSD_15m.h5
🚀 Treinando LSTM com múltiplas features para ETH-USD (15m)...


💾 Modelo LSTM salvo em: /content/models/lstm_model_ETHUSD_15m.h5
📦 Metadados salvos em: /content/models/lstm_model_ETHUSD_15m_meta.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔧 Calibração dinâmica aplicada: ajuste médio $-1082.72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [ ]:
run_analysis(
    selected_timeframes=[{"interval": "1d", "period": "1000d", "atr": 0.05}],
    plot_timeframes=["1d"],
    alert_timeframes=["1d"],
    retrain_models = True
)


In [23]:
import pandas as pd

# Recria o prediction_log.csv com as colunas padrão
from pathlib import Path

def resetar_prediction_log(filepath="/content/prediction_log.csv"):
    colunas = [
        "Asset", "Timeframe", "Date", "Price", "Signal", "Confidence", "AdjustedProb",
        "TP1", "TP2", "SL", "Accuracy", "Precision", "Recall", "F1",
        "LSTM_Predicted", "TargetPrice", "Acertou", "Resultado", "PrecoSaida", "LucroEstimado", "DuracaoMin"
    ]
    df_vazio = pd.DataFrame(columns=colunas)
    df_vazio.to_csv(filepath, index=False)
    print(f"✅ Log resetado: {filepath}")

resetar_prediction_log()


✅ Log resetado: /content/prediction_log.csv


In [ ]:
clear_models("/content/models")

🧹 Limpando todos os modelos salvos em: /content/models
✅ Modelos deletados com sucesso.


In [ ]:
criar_prediction_log_padrao("/content/prediction_log.csv")


📄 Criando arquivo de log padrão em: /content/prediction_log.csv


In [ ]:
import os

def limpar_prediction_log(path="/content/prediction_log.csv"):
    if os.path.exists(path):
        try:
            os.remove(path)
            print(f"🧹 Arquivo de log deletado com sucesso: {path}")
        except Exception as e:
            print(f"❌ Erro ao deletar o arquivo de log: {e}")

limpar_prediction_log()


🧹 Arquivo de log deletado com sucesso: /content/prediction_log.csv


In [ ]:
import os
import glob

def limpar_model_results():
    arquivos = glob.glob("/content/model_results_*.csv")
    if not arquivos:
        print("📂 Nenhum arquivo model_results_*.csv encontrado.")
        return

    for arquivo in arquivos:
        try:
            os.remove(arquivo)
            print(f"🧹 Arquivo deletado: {arquivo}")
        except Exception as e:
            print(f"❌ Erro ao deletar {arquivo}: {e}")

    print("✅ Todos os arquivos model_results_*.csv foram removidos.")

limpar_model_results()

🧹 Arquivo deletado: /content/model_results_2025-04-11_12-09-08.csv
🧹 Arquivo deletado: /content/model_results_2025-04-11_12-02-41.csv
✅ Todos os arquivos model_results_*.csv foram removidos.


In [ ]:
def limpar_prediction_log(path="prediction_log.csv"):
    if not os.path.exists(path):
        print("⚠️ Arquivo de log não encontrado.")
        return

    df = pd.read_csv(path)
    df = df.dropna(subset=["Date"])
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df[df["Date"].dt.year >= 2000]
    df.to_csv(path, index=False)
    print("✅ Log limpo com sucesso. Entradas de 1970 removidas!")

limpar_prediction_log()



✅ Log limpo com sucesso. Entradas de 1970 removidas!


In [ ]:
import os
os.remove("/content/prediction_log.csv")

In [ ]:
import os
if os.path.exists("prediction_log.csv"):
    os.remove("prediction_log.csv")
    print("🧹 Carteira virtual resetada com sucesso.")


In [ ]:
df = pd.read_csv("prediction_log.csv")
print(df.head(2).to_string())


In [ ]:
!rm -f /content/prediction_log.csv


In [ ]:
run_analysis(
    selected_timeframes=[{"interval": "15m", "period": "30d", "atr": 0.02}],
    plot_timeframes=["15m"],
    alert_timeframes=["15m"],
    retrain_models=True  # ⬅️ importante!
)


In [ ]:
simular_todos_trades("prediction_log.csv", df_candles=df, timeframe="15m")


NameError: name 'df' is not defined